# Ballchasing.com data

In [42]:
import requests
import pandas as pd
import pprint
import datetime as dt

UsageError: Line magic function `%%` not found.


In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
#get data from rlcs 2022-2023 season
url = "https://ballchasing.com/api/groups/swiss-8felo6us4y"

#url = "https://ballchasing.com/api/"
headers = {"Authorization": "DSycYDVcneyGEEl1MdHyM5JboSDBKVwyn8vJJ2WL"}

response = requests.get(url, headers=headers).json()
print(response)


{'id': 'swiss-8felo6us4y', 'link': 'https://ballchasing.com/api/groups/swiss-8felo6us4y', 'name': 'Swiss', 'created': '2022-11-07T18:38:11.917728Z', 'status': 'ok', 'player_identification': 'by-id', 'team_identification': 'by-distinct-players', 'shared': False, 'creator': {'steam_id': '76561199225615730', 'name': 'RLCS Referee', 'profile_url': 'https://steamcommunity.com/profiles/76561199225615730/', 'avatar': 'https://avatars.steamstatic.com/f84a836e794362eb3c53d50ac7893daa04c53c3d.jpg', 'avatar_full': '', 'avatar_medium': ''}, 'players': [{'platform': 'Steam', 'id': '76561198153434719', 'name': 'AZTRMICK', 'team': 'THE CLUB', 'cumulative': {'games': 16, 'wins': 7, 'win_percentage': 43.75, 'play_duration': 5915, 'core': {'shots': 37, 'shots_against': 124, 'goals': 10, 'goals_against': 26, 'saves': 21, 'assists': 5, 'score': 4630, 'mvp': 0, 'shooting_percentage': 27.027027}, 'boost': {'bpm': 417.9381, 'bcpm': 401.5486, 'avg_amount': 51.379997, 'amount_collected': 39586, 'amount_stolen'

{'created': '2022-11-07T18:38:11.917728Z',
 'creator': {'avatar': 'https://avatars.steamstatic.com/f84a836e794362eb3c53d50ac7893daa04c53c3d.jpg',
             'avatar_full': '',
             'avatar_medium': '',
             'name': 'RLCS Referee',
             'profile_url': 'https://steamcommunity.com/profiles/76561199225615730/',
             'steam_id': '76561199225615730'},
 'id': 'swiss-8felo6us4y',
 'link': 'https://ballchasing.com/api/groups/swiss-8felo6us4y',
 'name': 'Swiss',
 'player_identification': 'by-id',
 'players': [{'cumulative': {'boost': {'amount_collected': 39586,
                                       'amount_collected_big': 25558,
                                       'amount_collected_small': 14028,
                                       'amount_overfill': 8258,
                                       'amount_overfill_stolen': 1104,
                                       'amount_stolen': 9104,
                                       'amount_stolen_big': 5029,
   

Just realised I can get csv files of team stats in game by game format. Will be starting off with all regional main events of NA and EU regions (3 regionals for 3 splits) as well as fall and winter majors (they didnt have the spring major :/)

## Data Cleaning

### 1. Load data from CSVs folder

In [ ]:
winter_major_df = pd.read_csv('../ballchasing_csvs/winter_major_22-23.csv', sep=';',header=0)
winter_major_df

### 2. Remove Negligible columns
The columns removed were decided based on my game knowledge but these can be revisited later.

In [ ]:
winter_major_df.keys()

In [ ]:
winter_major_df = winter_major_df.drop(columns=['opposing team name','replay id', 'date', 'amount collected big pads',
       'amount collected small pads',
       'count collected small pads', 'amount stolen', 'amount stolen big pads',
       'amount stolen small pads', 'count stolen big pads',
       'count stolen small pads',
       'amount used while supersonic', 'amount overfill total',
       'amount overfill stolen', 'time slow speed',
       'time boost speed', 'time in front of ball.1', 'time defensive half', 'time offensive half',
       ])
winter_major_df

### 3. Split and Merge  
Every match is documented twice, once for each teams stats,  split, label and merge so we have one match per row with both teams' stats labeled team_1 and team_2 

In [92]:
winter_major_df_team1 = winter_major_df.iloc[0::2,:]
winter_major_df_team2 = winter_major_df.iloc[1::2,:] 

In [ ]:
# only run this once lmao 
winter_major_df_team2 = winter_major_df_team2.add_suffix('_2')
winter_major_df_team1 = winter_major_df_team1.add_suffix('_1')
winter_major_df_team1.rename(columns={'replay title_1': 'replay title'}, inplace=True)
winter_major_df_team2.rename(columns={'replay title_2': 'replay title'}, inplace=True)
winter_major_df_team2


## Testing final function


In [38]:
def get_event_df(split, event, season, region=None):
    if region:
        event_df = pd.read_csv(f'../ballchasing_csvs/{str(split)}_{str(event)}_{str(region)}_{str(season)}.csv', sep=';',header=0)
    else:
        event_df = pd.read_csv(f'../ballchasing_csvs/{str(split)}_{str(event)}_{str(season)}.csv', sep=';',header=0)
    #remove unnecessary/redundant columns
    event_df = event_df.drop(columns=['replay id', 'opposing team name', 'amount collected big pads',
       'amount collected small pads',
       'count collected small pads', 'amount stolen', 'amount stolen big pads',
       'amount stolen small pads', 'count stolen big pads',
       'count stolen small pads',
       'amount used while supersonic', 'amount overfill total',
       'amount overfill stolen', 'time slow speed',
       'time boost speed', 'time in front of ball.1', 'time defensive half', 'time offensive half',
       ])
    # convert date column to datetime and sort values by date
    event_df['date'] = pd.to_datetime(event_df['date'], yearfirst=True, format="%Y-%m-%d %H:%M:%S")
    event_df.sort_values(by='date', inplace=True)
    #Split, label and merge rows 2 by 2 for each match
    event_df_1 = event_df.iloc[0::2,:]
    event_df_2 = event_df.iloc[1::2,:]
    event_df_2 = event_df_2.drop(columns=['map', 'result', 'replay title'])
    event_df_2 = event_df_2.add_suffix('_2')
    event_df_1 = event_df_1.add_suffix('_1')
    event_df_1.rename(columns={'replay title_1': 'replay title', 'map_1': 'map', 'date_1':'date'}, inplace=True)
    event_df_2.rename(columns={'date_2': 'date'}, inplace=True)
    event_df_merged = event_df_1.merge(event_df_2, 'outer', 'date')
    return event_df_merged

In [104]:
spring_reg2_EU_df = get_event_df('spring', 'reg2', '22-23', 'EU')
spring_reg2_EU_df

In [168]:
reg_list = ['reg1', 'reg2', 'reg3']

def get_split_regionals_df(split, season, region):
    reg_dfs = [get_event_df(split, reg, season, region) for reg in reg_list]
    regional_split_df = pd.concat(reg_dfs)
    return regional_split_df


In [ ]:
winter_split_reg_df = get_split_regionals_df('winter', '22-23', 'EU')
winter_split_reg_df

In [141]:
regions = 'EU'
type(regions)

if type(regions) == str:
    print('issa string')

issa string


In [169]:
def get_split_with_major(split, season, regions):
    major_df = get_event_df(split, 'major', season)
    if type(regions) == str:
        split_regionals_df = get_split_regionals_df(split, season, regions)
        split_with_major_dfs = [split_regionals_df, major_df]
        split_with_major_df = pd.concat(split_with_major_dfs)
        return split_with_major_df
    else:
        split_regionals_dfs = [get_split_regionals_df(split, season, region) for region in regions]
        split_regionals_dfs.append(major_df)
        splits_with_major_df = pd.concat(split_regionals_dfs)
        return splits_with_major_df

In [ ]:
eu_split_with_major = get_split_with_major('fall', '22-23', 'EU')
eu_split_with_major

In [171]:
split_list = ['winter', 'fall',] #spring major 22-23 hasn't happened yet

def get_majors(season):
    major_dfs_list = [get_event_df(split, 'major', season) for split in split_list]
    majors_df = pd.concat(major_dfs_list)
    return majors_df

In [24]:
majors_23 = get_majors('22-23')
majors_23

NameError: name 'get_majors' is not defined

Ok so I've realised I missed a few things for the dfs. Firstly, I want the predictions To take the most recent games into account and give them a slightly higher weight. to do this I will need to potentially add a a match order within the date, although this will be quite long winded since RLCS events are played in a "Swiss" format involving upper and lower brackets and bracket resets etc with series varying between Bo5 and Bo7.

## Add date column

In [40]:
winter_major_df = get_event_df('winter', 'major', '22-23')
winter_major_df

,replay title,map,date,result_1,team name_1,score_1,goals_1,assists_1,saves_1,shots_1,...,time high in air_2,time powerslide_2,count powerslide_2,time in front of ball_2,time behind ball_2,time defensive third_2,time neutral third_2,time offensive third_2,demos inflicted_2,demos taken_2
0,EU T-A GA vs G1 G1 2023-02-26.13.41,Mannfield (Night),2023-02-26 19:41:01,win,G1,1199,2,0,6,6,...,55.14,26.93,257,292.40,750.02,432.03,326.11,284.29,2,2
1,EU T-A GA vs G1 G2 2023-02-26.13.48,Forbidden Temple,2023-02-26 19:48:54,win,GERMAN AMIGOS,1140,3,2,2,13,...,68.83,24.02,213,291.70,753.64,631.70,267.84,145.78,1,5
2,EU T-A GA vs G1 G3 2023-02-26.13.55,DFH Stadium,2023-02-26 19:55:46,win,G1,1168,2,1,7,9,...,50.71,24.96,244,297.35,688.80,381.72,327.50,276.94,3,3
3,EU T-A GA vs G1 G4 2023-02-26.14.03,Utopia Coliseum (Dusk),2023-02-26 20:03:38,loss,G1,1055,1,1,8,4,...,66.00,28.01,255,233.22,781.35,444.74,320.62,249.19,2,4
4,EU T-A GA vs G1 G5 2023-02-26.14.12,"Wasteland (Standard, Night)",2023-02-26 20:12:19,win,G1,1076,2,1,5,7,...,60.87,24.33,242,266.89,731.59,429.69,362.06,206.72,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,WM P-K FaZe vs KC G2 2023-04-09.16.40,Forbidden Temple,2023-04-09 23:40:34,loss,KARMINE CORP,668,0,0,4,4,...,64.49,21.46,189,227.01,749.81,447.08,312.42,217.32,5,2
163,WM P-K FaZe vs KC G3 2023-04-09.16.47,DFH Stadium,2023-04-09 23:47:59,loss,KARMINE CORP,693,0,0,4,11,...,63.22,18.63,167,255.47,687.37,476.59,279.20,187.05,1,3
164,WM P-K FaZe vs KC G4 2023-04-09.16.57,Utopia Coliseum (Dusk),2023-04-09 23:57:51,loss,FAZE CLAN,954,0,0,8,3,...,75.62,23.41,207,220.72,869.75,461.37,362.11,266.99,1,2
165,WM P-K FaZe vs KC G5 2023-04-09.17.09,"Wasteland (Standard, Night)",2023-04-10 00:09:08,win,KARMINE CORP,1121,3,3,4,9,...,70.83,22.39,206,286.30,732.04,500.85,325.69,191.81,6,5


In [47]:
# Convert win / loss to 0 / 1
winter_major_df['result_1'] = winter_major_df['result_1'].map({'win': 1, 'loss': 0})
winter_major_df


,replay title,map,date,result_1,team name_1,score_1,goals_1,assists_1,saves_1,shots_1,...,time high in air_2,time powerslide_2,count powerslide_2,time in front of ball_2,time behind ball_2,time defensive third_2,time neutral third_2,time offensive third_2,demos inflicted_2,demos taken_2
0,EU T-A GA vs G1 G1 2023-02-26.13.41,Mannfield (Night),2023-02-26 19:41:01,1,G1,1199,2,0,6,6,...,55.14,26.93,257,292.40,750.02,432.03,326.11,284.29,2,2
1,EU T-A GA vs G1 G2 2023-02-26.13.48,Forbidden Temple,2023-02-26 19:48:54,1,GERMAN AMIGOS,1140,3,2,2,13,...,68.83,24.02,213,291.70,753.64,631.70,267.84,145.78,1,5
2,EU T-A GA vs G1 G3 2023-02-26.13.55,DFH Stadium,2023-02-26 19:55:46,1,G1,1168,2,1,7,9,...,50.71,24.96,244,297.35,688.80,381.72,327.50,276.94,3,3
3,EU T-A GA vs G1 G4 2023-02-26.14.03,Utopia Coliseum (Dusk),2023-02-26 20:03:38,0,G1,1055,1,1,8,4,...,66.00,28.01,255,233.22,781.35,444.74,320.62,249.19,2,4
4,EU T-A GA vs G1 G5 2023-02-26.14.12,"Wasteland (Standard, Night)",2023-02-26 20:12:19,1,G1,1076,2,1,5,7,...,60.87,24.33,242,266.89,731.59,429.69,362.06,206.72,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,WM P-K FaZe vs KC G2 2023-04-09.16.40,Forbidden Temple,2023-04-09 23:40:34,0,KARMINE CORP,668,0,0,4,4,...,64.49,21.46,189,227.01,749.81,447.08,312.42,217.32,5,2
163,WM P-K FaZe vs KC G3 2023-04-09.16.47,DFH Stadium,2023-04-09 23:47:59,0,KARMINE CORP,693,0,0,4,11,...,63.22,18.63,167,255.47,687.37,476.59,279.20,187.05,1,3
164,WM P-K FaZe vs KC G4 2023-04-09.16.57,Utopia Coliseum (Dusk),2023-04-09 23:57:51,0,FAZE CLAN,954,0,0,8,3,...,75.62,23.41,207,220.72,869.75,461.37,362.11,266.99,1,2
165,WM P-K FaZe vs KC G5 2023-04-09.17.09,"Wasteland (Standard, Night)",2023-04-10 00:09:08,1,KARMINE CORP,1121,3,3,4,9,...,70.83,22.39,206,286.30,732.04,500.85,325.69,191.81,6,5
